In [1]:
import pandas as pd
import numpy as np
import os


In [2]:

# Função para carregar o arquivo CSV
def load_data(file_path):
    try:
        # Lê o arquivo a partir da terceira linha (pulando as duas primeiras)
        df = pd.read_csv(
            file_path, 
            delimiter=';', 
            decimal=',', 
            skipinitialspace=True, 
            encoding='ISO-8859-1',
            skiprows=1  # Ignorar as 2 primeiras linhas
        )
        
        # Verificar se o número de colunas bate com a renomeação
        if len(df.columns) == 8:  # Substitua pelo número esperado de colunas
            df.columns = [
                'Tempo (s)', 'Deslocamento (mm)', 'Força (kN)', 
                'Deformação à tração (%)', 'Deslocamento à tração (mm)', 
                'PIP', 'Tenacidade (N/tex)', 'Tensão à tração (MPa)'
            ]
        else:
            print("Número de colunas no CSV não corresponde ao esperado.")
            print(f"Colunas encontradas: {df.columns}")
            return None
        
        # Converte apenas as colunas numéricas para float
        colunas_numericas = [
            'Tempo (s)', 'Deslocamento (mm)', 'Força (kN)', 
            'Deformação à tração (%)', 'Deslocamento à tração (mm)', 
            'Tenacidade (N/tex)', 'Tensão à tração (MPa)'
        ]
        
        for col in colunas_numericas:
            if col in df.columns:  # Verifica se a coluna existe antes de processar
                # Verifica se a coluna é do tipo 'object' (string) antes de usar .str.replace
                if df[col].dtype == 'object':
                    df[col] = df[col].str.replace(',', '.', regex=False)
                # Converte para numérico
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        return df

    except Exception as e:
        print(f"Erro ao carregar o arquivo: {e}")
        return None

# Caminho para o arquivo CSV
file_path = r'../Lab1/PLA FIBRA DE CARBONO/PLA FIBRA DE CARBONO_4.csv'

# Carregar os dados
df = load_data(file_path)

# Ver os primeiros registros para conferir
if df is not None:
    print(df.head())
else:
    print("Erro ao carregar os dados.")


   Tempo (s)  Deslocamento (mm)  Força (kN)  Deformação à tração (%)  \
0       0.00             0.0006      0.0000                   0.0000   
1       0.02             0.0013      0.0007                   0.0003   
2       0.04             0.0068      0.0064                   0.0032   
3       0.06             0.0168      0.0166                   0.0085   
4       0.08             0.0287      0.0285                   0.0148   

   Deslocamento à tração (mm)  PIP  Tenacidade (N/tex)  Tensão à tração (MPa)  
0                      0.0000  0.0             -0.0042                -0.0001  
1                      0.0007  0.0              0.7398                 0.0123  
2                      0.0062  0.0              6.3981                 0.1066  
3                      0.0162  0.0             16.5819                 0.2764  
4                      0.0281  0.0             28.5262                 0.4754  


In [31]:
df 

,Tempo (s),Deslocamento (mm),Força (kN),Deformação à tração (%),Deslocamento à tração (mm),PIP,Tenacidade (N/tex),Tensão à tração (MPa)
0,0.0000,0.0006,0.0000,0.0000,0.0000,0.0,-0.0042,-0.0001
1,0.0200,0.0013,0.0007,0.0003,0.0007,0.0,0.7398,0.0123
2,0.0400,0.0068,0.0064,0.0032,0.0062,0.0,6.3981,0.1066
3,0.0600,0.0168,0.0166,0.0085,0.0162,0.0,16.5819,0.2764
4,0.0800,0.0287,0.0285,0.0148,0.0281,0.0,28.5262,0.4754
...,...,...,...,...,...,...,...,...
299,5.3622,8.4096,0.8365,4.4258,8.4090,0.0,836.4558,13.9409
300,5.3626,8.4202,0.8211,4.4313,8.4196,0.0,821.0710,13.6845
301,5.3630,8.4307,0.8057,4.4369,8.4301,0.0,805.6874,13.4281
302,5.3634,8.4413,0.7903,4.4425,8.4407,0.0,790.3128,13.1719


In [3]:
# 1 num_pontos= 17.84 / 0.02   num_pontos=892
# 2 num_pontos= 2.82 / 0.02    num_pontos=141
# 3 num_pontos= 17.42 / 0.02   num_pontos=871
# 4 num_pontos= 2.93 / 0.02    num_pontos=146


from scipy import stats


def calc_modulo_elasticidade_dinamico(df, limite_erro=0.01): #como entrada um DataFrame df e um limite_erro, que determina quão próximo da linearidade os dados devem estar.
    # Filtra valores de tensão positivos
    filtro = df[df['Tensão à tração (MPa)'] > 0]

    # Inicializa variáveis
    n_max = len(filtro)
    modulo_elasticidade = np.nan

    for n in range(2, n_max):  # Começar com 2 pontos
        tensao = filtro['Tensão à tração (MPa)'].iloc[:n].values      #Para cada iteração do loop, os dados de tensão e deformação são extraídos até o índice n. A deformação é convertida de porcentagem para fração dividindo por 100.
        deformacao = filtro['Deformação à tração (%)'].iloc[:n].values / 100

        # Verifica se todos os valores de deformação ou tensão são iguais
        if np.all(deformacao == deformacao[0]) or np.all(tensao == tensao[0]):
            continue  # Ignora este conjunto e passa para o próximo

        # Realiza a regressão linear
        slope, intercept, r_value, p_value, std_err = stats.linregress(deformacao, tensao) #Realiza a regressão linear nos dados de tensão e deformação. 
                                                                                           #slope representa o coeficiente angular da linha de melhor ajuste, 
                                                                                           # que é o que queremos calcular como o módulo de elasticidade.

        # Prediz a tensão usando a linha de melhor ajuste
        tensao_predita = slope * deformacao + intercept

        # Prediz os valores de tensão com base na linha de melhor ajuste e calcula o Erro Quadrático Médio (EQM) entre os valores reais e os preditos.
        eqm = np.mean((tensao - tensao_predita) ** 2)


#Se o EQM for menor que o limite definido, atualiza modulo_elasticidade com o valor do coeficiente angular (slope).
        # Verifica se o EQM é menor que o limite
        if eqm < limite_erro:
            modulo_elasticidade = slope  # O coeficiente angular é o módulo de elasticidade
        else:
            break  # Sai do loop se a linearidade não se mantiver

    return modulo_elasticidade

# Exemplo de uso
modulo_elasticidade_dinamico = calc_modulo_elasticidade_dinamico(df)

if not np.isnan(modulo_elasticidade_dinamico):
    print(f"Módulo de Elasticidade (dinâmico): {modulo_elasticidade_dinamico:.2f} MPa")
else:
    print("Não foi possível calcular o módulo de elasticidade.")

Módulo de Elasticidade (dinâmico): 2707.55 MPa


In [4]:
# Função para calcular a resistência máxima e a deformação correspondente
def calc_resistencia_maxima(df):
    # Verifica se o DataFrame contém dados suficientes
    if df.empty or 'Tensão à tração (MPa)' not in df.columns:
        return np.nan, np.nan, np.nan  # Retorna NaN se não houver dados suficientes

    # Encontra a resistência máxima e a deformação correspondente
    resistencia_maxima = df['Tensão à tração (MPa)'].max()  # Encontra a máxima tensão
    idx_max = df['Tensão à tração (MPa)'].idxmax()  # Índice da máxima tensão
    deformacao_maxima_percent = df.loc[idx_max, 'Deformação à tração (%)']  # Deformação correspondente à tensão máxima
    deformacao_maxima_mm = df.loc[idx_max, 'Deslocamento à tração (mm)']  # Deformação em mm

    return resistencia_maxima, deformacao_maxima_percent, deformacao_maxima_mm

# Cálculo da resistência máxima e deformações correspondentes
resistencia_maxima, deformacao_maxima, deformacao_maxima_mm = calc_resistencia_maxima(df)

# Resultados
print(f"Resistência Máxima: {resistencia_maxima:.2f} MPa")
print(f"Deformação na Resistência Máxima: {deformacao_maxima:.2f} %")
print(f"Deformação na Resistência Máxima (mm): {deformacao_maxima_mm:.2f} mm")

Resistência Máxima: 24.35 MPa
Deformação na Resistência Máxima: 1.64 %
Deformação na Resistência Máxima (mm): 3.12 mm


In [5]:
def calc_resistencia_ruptura(df):
    # Verifica se o DataFrame contém dados suficientes
    if df.empty or 'Tensão à tração (MPa)' not in df.columns:
        return np.nan, np.nan  # Retorna NaN se não houver dados suficientes

    # A resistência na ruptura é a última tensão registrada no DataFrame
    resistencia_ruptura = df['Tensão à tração (MPa)'].iloc[-1]  # Última tensão
    deformacao_ruptura = df['Deslocamento à tração (mm)'].iloc[-1]  # Último valor de deslocamento

    return resistencia_ruptura, deformacao_ruptura

# Cálculos
resistencia_ruptura, deformacao_ruptura_mm = calc_resistencia_ruptura(df)

# Resultados
print(f"Resistência na Ruptura: {resistencia_ruptura:.2f} MPa")
print(f"Deformação na Ruptura: {deformacao_ruptura_mm:.2f} mm")


Resistência na Ruptura: 13.04 MPa
Deformação na Ruptura: 8.45 mm


In [6]:
def calc_tenacidade_media(df):
    # Verifica se o DataFrame contém dados suficientes
    if df.empty or 'Tensão à tração (MPa)' not in df.columns or 'Deformação à tração (%)' not in df.columns:
        return np.nan  # Retorna NaN se não houver dados suficientes

    # Calcular a tenacidade como a área sob a curva tensão-deformação
    # Convertendo a tensão de MPa para kgf/mm (1 MPa = 0.10197 kgf/mm²)
    tensao_kgf_mm = df['Tensão à tração (MPa)'] * 0.10197
    deformacao_fraçao = df['Deformação à tração (%)'] / 100  # Convertendo a deformação para fração

    # Cálculo da tenacidade como a área sob a curva
    tenacidade = np.trapezoid(tensao_kgf_mm, deformacao_fraçao)  # Área sob a curva

    return tenacidade


# Cálculos
tenacidade_media = calc_tenacidade_media(df)

# Resultados
print(f"Tenacidade Média: {tenacidade_media:.2f} kgf/mm")

Tenacidade Média: 0.09 kgf/mm


In [7]:
def calc_tenacidade_j_m3(df):
    
    # Convertendo a tensão de MPa para Pa (1 MPa = 1e6 Pa)
    tensao_pa = df['Tensão à tração (MPa)'] * 1e6
  

    # Convertendo a deformação de porcentagem para fração
    deformacao_fraçao = df['Deformação à tração (%)'] / 100  
   

    # Cálculo da tenacidade como a área sob a curva
    tenacidade = np.trapezoid(tensao_pa, deformacao_fraçao)  # Área sob a curva

    return tenacidade

tenacidade_j_m3 = calc_tenacidade_j_m3(df)

# Resultados
print(f"Tenacidade: {tenacidade_j_m3:.2f} J/m³")

Tenacidade: 899202.99 J/m³
